本程式想要根據日本的地址來預測該地址的郵遞區號
<br>輸入 : 北海道帯広市西１２条北１丁目８
<br>輸出 : 〒080-0042(實際的輸出只會有數字的部分→0800042)　
<br>

郵遞區號每一個位數都是[0 , 1 , 2 , 3 , 4 , 5 , 6 , 7 , ,8 , 9]中的其中一個數字
<br>若用傳統的多分類問題的手法來分析這個case，那出輸出的可能性將呈現指數級增長，我們不太可能直接這樣幹
<br>所以需要對每一個位數分別預測是屬於0~9中的哪一個class
而這個case即屬於Multi-Label的問題

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import tensorflow as tf
import math

In [2]:
vocabulary = []
f = open('addresses4.txt' , 'rb')
data = f.readlines()[:100000] # 取前100000筆來訓練
address_ = []
postal_ = [] 
for line in data:
    postal , address = line.decode('utf8').strip().split('　')
    address_.append(address)
    postal_split = postal[1:].split('-')[0] + postal[1:].split('-')[1]
    postal_temp = []
    for i in postal_split:
        postal_temp.append(int(i)) # 每個postal都只有 7 個數字
    postal_.append(postal_temp)
    for element in list(postal):
        vocabulary.append(element)
    for element in list(address):
        vocabulary.append(element)

vocabulary = sorted(list(set(vocabulary)))

In [3]:
word_to_int = {}
for i , word in enumerate(vocabulary):
    word_to_int[word] = i

word_to_int['<PAD>'] = i + 1 # 將'<PAD>'也列入詞庫 
int_to_word = dict(zip(word_to_int.values() , word_to_int.keys()))   

max_len = 0 # 取得地址最大長度
for sentence in address_:
    max_len = max(max_len , len(sentence))

In [4]:
address_encode = [] # 將地址換成index
for sentence in address_:
    temp = []
    for word in sentence:
        temp.append(word_to_int[word])
    while len(temp) < max_len: # 若地址小於最大長度則補 word_to_int['pad']
        temp.append(word_to_int['<PAD>'])
    address_encode.append(temp) 

    
address_encode = np.array(address_encode)
postal_ = np.array(postal_)

# 因為地址是照地區排序的，若沒有做shuffle這個動作，有些地區的地址會在training的過程中完全沒見過，導致最後testing時無法預測
indices = np.arange(len(address_encode))
np.random.shuffle(indices)
address_encode = address_encode[indices]
postal_ = postal_[indices]

split_at = len(address_) - len(address_) // 10
address_train , address_val = address_encode[:split_at] , address_encode[split_at:]
postal_train , postal_val = postal_[:split_at] , postal_[split_at:]

In [5]:
# 超參數
rnn_size = 256
batch_size = 300
vocab_size = len(word_to_int)
embedding_size = 200

In [6]:
sources = tf.placeholder(tf.int32 , [None , max_len] , name = 'sources')
targets = tf.placeholder(tf.int64 , [None , 7] , name = 'targets')
targets_onehot = tf.one_hot(tf.reshape(targets , [-1]) , depth = 10)

embeddings = tf.Variable(tf.random_uniform([vocab_size , embedding_size], -1, 1))
embed_input = tf.nn.embedding_lookup(embeddings , sources)

def lstm_cell(hidden_size): 
    cell = tf.contrib.rnn.LSTMCell(hidden_size)
    return cell


with tf.variable_scope('LSTM'):
    mlstm_cell = tf.contrib.rnn.MultiRNNCell([lstm_cell(rnn_size) for _ in range(1)] ,  # 總共1層rnn
                                              state_is_tuple = True)
    
    outputs , states = tf.nn.dynamic_rnn(mlstm_cell , 
                                         inputs = embed_input ,
                                         dtype = tf.float32) # outputs → [batch_size , max_len , rnn_size]


outputs_ = tf.reshape(outputs , [-1 , max_len * rnn_size]) # [batch_size , max_len * rnn_size]
# 在最後一層加上一個全連接層
weights = tf.Variable(tf.truncated_normal([max_len * rnn_size , 7 * 10] , mean = 0.01 , stddev = 0.1))
biases = tf.Variable(tf.zeros([1 , 7 * 10]) + 0.0001)
logits = tf.matmul(outputs_ , weights) + biases  # [batch_size , 7 * 10]
logits = tf.reshape(logits , [-1 , 10]) # [batch_size * 7 , 10]

In [7]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits , labels = targets_onehot))
train_op = tf.train.AdamOptimizer(0.001).minimize(cost)

prediction = tf.reshape(logits , [-1 , 7 , 10]) # prediction → [batch_size , 7 , 10]
prediction = tf.argmax(prediction , axis = 2)   # prediction → [batch_size , 7]

'''
prediction:[1 2 3 5 5 6 7      targets:[1 2 3 4 5 6 7
            2 3 4 5 6 6 8]              2 3 4 5 6 7 8]

tf.equal(prediction , targets) → [True True True False True True True    → correct = [1 1 1 0 1 1 1
                                   True True True True  True True True]                 1 1 1 1 1 1 1]

tf.reduce_min(correct , axis = 1) → [0
                                      1]

只要prediction中1個數字錯，就相當於全錯
'''
correct = tf.cast(tf.equal(prediction , targets) , tf.float32)
accuracy = tf.reduce_mean(tf.reduce_min(correct , axis = 1))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [8]:
# 決定每個batchy訓練資料的index
step = (math.ceil(len(address_train) / batch_size)) * batch_size
temp = []
j = 0
index = []
for ii in range(0 , step):
    j = j + 1
    if j > len(address_train):
        j = j - (len(address_train))   
    temp.append(j)  
    if len(temp) == batch_size:
        index.append(temp)
        temp = []

index = list(np.array(index) - 1)

In [9]:
sess = tf.Session() 
sess.run(tf.global_variables_initializer())


batch_loss_record = []
batch_accuracy_record = []
batch_loss_mean_record = []
batch_accuracy_mean_record = []
val_loss_record = []
val_accuracy_record = []
for epoch_i in range(0 , 3):        
    for batch_i in range(0 , len(index)):        
        batch_source = address_train[index[batch_i] , :]
        batch_target = postal_train[index[batch_i] , :]
        
        _ , batch_loss , batch_accuracy , prediction_result =\
        sess.run([train_op , cost , accuracy , prediction] , 
                 feed_dict = {sources : batch_source , 
                              targets : batch_target})
        
        batch_loss_record.append(batch_loss)
        batch_accuracy_record.append(batch_accuracy)        
        if len(batch_loss_record) == 20:
            batch_loss_mean = np.array(batch_loss_record).mean()
            batch_accuracy_mean = np.array(batch_accuracy_record).mean()    
            batch_loss_mean_record.append(batch_loss_mean) 
            batch_accuracy_mean_record.append(batch_accuracy_mean)
            batch_loss_record = []
            batch_accuracy_record = []
              
        print('\nepoch : {} , batch : {} , loss : {:.2f}'.format(epoch_i , batch_i , batch_loss))
        print('epoch : {} , batch : {} , accuracy : {:.2%}'.format(epoch_i , batch_i , batch_accuracy))
        
        data_index = np.random.randint(batch_size)  
        
        source_visualization = []
        for i in batch_source[data_index]:
            if int_to_word[i] == '<PAD>': break
            source_visualization.append(int_to_word[i])
        source_visualization = ' '.join(source_visualization)
        
        target_visualization = []
        for i in batch_target[data_index]:
            target_visualization.append(str(i))
        target_visualization = ' '.join(target_visualization)

        predict_visualization = []
        for i in prediction_result[data_index]:
            predict_visualization.append(str(i))
        predict_visualization = ' '.join(predict_visualization)
        
        print('Source : {}'.format(source_visualization))
        print('Target : {}'.format(target_visualization))
        print('Predict : {}\n'.format(predict_visualization))
        
        if batch_i % 20 == 0 and batch_i > 0:
            batch_val_source = address_val
            batch_val_target = postal_val
            val_loss , val_accuracy = sess.run([cost , accuracy] , 
                                               feed_dict = {sources : batch_val_source , 
                                                            targets : batch_val_target})
            
            val_loss_record.append(val_loss) 
            val_accuracy_record.append(val_accuracy)                                                   
            
            print('\n' + '=' * 30)
            print('epoch : {} , val loss : {:.2f}'.format(epoch_i , val_loss))
            print('epoch : {} , val accuracy : {:.2%}'.format(epoch_i , val_accuracy))
            print('=' * 30 + '\n')


epoch : 0 , batch : 0 , loss : 3.19
epoch : 0 , batch : 0 , accuracy : 0.00%
Source : 愛 知 県 名 古 屋 市 昭 和 区 福 江 ２ 丁 目 ４ − １ ５
Target : 4 6 6 0 0 5 9
Predict : 5 9 4 2 9 5 9


epoch : 0 , batch : 1 , loss : 2.14
epoch : 0 , batch : 1 , accuracy : 0.00%
Source : 宮 城 県 登 米 市 米 山 町 西 野 字 今 平 １ １ ６
Target : 9 8 7 0 3 2 1
Predict : 8 1 3 0 0 1 1


epoch : 0 , batch : 2 , loss : 2.10
epoch : 0 , batch : 2 , accuracy : 0.00%
Source : 岩 手 県 大 船 渡 市 日 頃 市 町 字 長 安 寺 ９ ２ − ６
Target : 0 2 2 0 0 0 5
Predict : 9 2 9 0 0 4 4


epoch : 0 , batch : 3 , loss : 1.95
epoch : 0 , batch : 3 , accuracy : 0.00%
Source : 福 岡 県 直 方 市 大 字 頓 野 ２ １ ０ ４ − ３ １
Target : 8 2 2 0 0 0 2
Predict : 7 2 9 0 0 1 4


epoch : 0 , batch : 4 , loss : 1.90
epoch : 0 , batch : 4 , accuracy : 0.00%
Source : 東 京 都 大 田 区 池 上 １ 丁 目 ３ ２ − １ ２
Target : 1 4 6 0 0 8 2
Predict : 1 4 6 0 0 4 4


epoch : 0 , batch : 5 , loss : 1.78
epoch : 0 , batch : 5 , accuracy : 1.00%
Source : 宮 城 県 岩 沼 市 押 分 字 北 土 手 ８ ３
Target : 9 8 9 2 4 2 3
Predict : 9


epoch : 0 , batch : 48 , loss : 0.72
epoch : 0 , batch : 48 , accuracy : 26.00%
Source : 沖 縄 県 島 尻 郡 八 重 瀬 町 字 伊 覇 ２ ８ ４ − ３ ０ ２
Target : 9 0 1 0 4 0 5
Predict : 9 0 1 0 5 1 1


epoch : 0 , batch : 49 , loss : 0.71
epoch : 0 , batch : 49 , accuracy : 21.00%
Source : 愛 知 県 名 古 屋 市 昭 和 区 山 里 町 １ ８
Target : 4 6 6 0 8 2 4
Predict : 4 6 6 0 8 0 4


epoch : 0 , batch : 50 , loss : 0.70
epoch : 0 , batch : 50 , accuracy : 22.00%
Source : 富 山 県 射 水 市 中 新 湊 ４ − ２ ２
Target : 9 3 4 0 0 2 7
Predict : 9 3 9 0 3 0 1


epoch : 0 , batch : 51 , loss : 0.75
epoch : 0 , batch : 51 , accuracy : 21.00%
Source : 東 京 都 港 区 西 麻 布 ３ 丁 目 ２ ０ − １ ６
Target : 1 0 6 0 0 3 1
Predict : 1 0 6 0 0 3 1


epoch : 0 , batch : 52 , loss : 0.69
epoch : 0 , batch : 52 , accuracy : 25.67%
Source : 島 根 県 松 江 市 宍 道 町 上 来 待 １ ４ ３
Target : 6 9 9 0 4 0 5
Predict : 6 2 2 0 0 0 5


epoch : 0 , batch : 53 , loss : 0.66
epoch : 0 , batch : 53 , accuracy : 30.33%
Source : 宮 城 県 登 米 市 南 方 町 西 山 成 前 １ １ １
Target : 9 8 7 0 4 0 1
Predict


epoch : 0 , batch : 95 , loss : 0.41
epoch : 0 , batch : 95 , accuracy : 42.67%
Source : 熊 本 県 球 磨 郡 多 良 木 町 大 字 多 良 木 １ ５ ７ ７ − ７
Target : 8 6 8 0 5 0 1
Predict : 8 6 8 0 5 0 1


epoch : 0 , batch : 96 , loss : 0.39
epoch : 0 , batch : 96 , accuracy : 50.33%
Source : 東 京 都 足 立 区 竹 の 塚 １ 丁 目 ２ ３ − ９
Target : 1 2 1 0 8 1 3
Predict : 1 2 1 0 8 1 3


epoch : 0 , batch : 97 , loss : 0.39
epoch : 0 , batch : 97 , accuracy : 48.00%
Source : 東 京 都 大 田 区 田 園 調 布 本 町 ４ １ − １ − ４ ０ １
Target : 1 4 5 0 0 7 2
Predict : 1 4 5 0 0 7 1


epoch : 0 , batch : 98 , loss : 0.36
epoch : 0 , batch : 98 , accuracy : 51.00%
Source : 福 岡 県 遠 賀 郡 遠 賀 町 大 字 島 津 ４ ９ ５ − ９
Target : 8 1 1 4 3 0 1
Predict : 8 1 1 4 3 0 2


epoch : 0 , batch : 99 , loss : 0.37
epoch : 0 , batch : 99 , accuracy : 53.33%
Source : 富 山 県 射 水 市 寺 塚 原 ８ ０ ９ − １
Target : 9 3 4 0 0 5 6
Predict : 9 3 4 0 0 2 1


epoch : 0 , batch : 100 , loss : 0.38
epoch : 0 , batch : 100 , accuracy : 51.67%
Source : 福 井 県 坂 井 市 春 江 町 中 筋 高 田 １ ６ − ８
Target


epoch : 0 , val loss : 0.25
epoch : 0 , val accuracy : 69.42%


epoch : 0 , batch : 141 , loss : 0.25
epoch : 0 , batch : 141 , accuracy : 68.33%
Source : 鹿 児 島 県 霧 島 市 国 分 中 央 ３ 丁 目 ７ − １ １ − １ Ｆ
Target : 8 9 9 4 3 3 2
Predict : 8 9 9 4 3 3 2


epoch : 0 , batch : 142 , loss : 0.26
epoch : 0 , batch : 142 , accuracy : 68.00%
Source : 鹿 児 島 県 鹿 児 島 市 荒 田 １ 丁 目 ４ − １
Target : 8 9 0 0 0 5 4
Predict : 8 9 0 0 0 5 4


epoch : 0 , batch : 143 , loss : 0.26
epoch : 0 , batch : 143 , accuracy : 67.00%
Source : 長 野 県 北 佐 久 郡 御 代 田 町 大 字 御 代 田 ２ ４ ２ ２ − ５ ２
Target : 3 8 9 0 2 0 6
Predict : 3 8 9 0 2 0 6


epoch : 0 , batch : 144 , loss : 0.23
epoch : 0 , batch : 144 , accuracy : 71.33%
Source : 埼 玉 県 草 加 市 草 加 １ 丁 目 ５ − ６
Target : 3 4 0 0 0 4 3
Predict : 3 4 0 0 0 4 3


epoch : 0 , batch : 145 , loss : 0.26
epoch : 0 , batch : 145 , accuracy : 66.33%
Source : 福 岡 県 直 方 市 大 字 頓 野 ７ ９ ８ − １
Target : 8 2 2 0 0 0 2
Predict : 8 2 2 0 0 0 2


epoch : 0 , batch : 146 , loss : 0.21
epoch : 0 , batch :


epoch : 0 , batch : 187 , loss : 0.16
epoch : 0 , batch : 187 , accuracy : 78.33%
Source : 東 京 都 足 立 区 竹 の 塚 ６ 丁 目 １ ４ − １
Target : 1 2 1 0 8 1 3
Predict : 1 2 1 0 8 1 3


epoch : 0 , batch : 188 , loss : 0.15
epoch : 0 , batch : 188 , accuracy : 83.00%
Source : 千 葉 県 柏 市 し い の 木 台 ５ 丁 目 ３ ５ − ２ ６
Target : 2 7 7 0 9 4 5
Predict : 2 7 7 0 9 4 5


epoch : 0 , batch : 189 , loss : 0.17
epoch : 0 , batch : 189 , accuracy : 79.33%
Source : 東 京 都 杉 並 区 永 福 ３ 丁 目 ５ ６ − ３ ０
Target : 1 6 8 0 0 6 4
Predict : 1 6 8 0 0 6 4


epoch : 0 , batch : 190 , loss : 0.18
epoch : 0 , batch : 190 , accuracy : 76.67%
Source : 大 阪 府 大 東 市 西 楠 の 里 町 ４ − ６
Target : 5 7 4 0 0 0 5
Predict : 5 7 4 0 0 0 7


epoch : 0 , batch : 191 , loss : 0.18
epoch : 0 , batch : 191 , accuracy : 74.33%
Source : 大 阪 府 大 阪 市 住 吉 区 南 住 吉 ３ 丁 目 １ ６ − １ ２ − １ ０ ５
Target : 5 5 8 0 0 4 1
Predict : 5 5 8 0 0 4 1


epoch : 0 , batch : 192 , loss : 0.15
epoch : 0 , batch : 192 , accuracy : 82.33%
Source : 福 岡 県 直 方 市 大 字 植 木 １ ７ １
Target

...


epoch : 2 , batch : 101 , loss : 0.01
epoch : 2 , batch : 101 , accuracy : 99.00%
Source : 兵 庫 県 神 戸 市 長 田 区 苅 藻 島 町 ３ 丁 目 ７ − １ ３
Target : 6 5 3 0 0 3 3
Predict : 6 5 3 0 0 3 3


epoch : 2 , batch : 102 , loss : 0.03
epoch : 2 , batch : 102 , accuracy : 97.00%
Source : 静 岡 県 焼 津 市 与 惣 次 ４ ８
Target : 4 2 5 0 0 3 4
Predict : 4 2 5 0 0 3 4


epoch : 2 , batch : 103 , loss : 0.01
epoch : 2 , batch : 103 , accuracy : 99.00%
Source : 山 口 県 山 口 市 小 郡 高 砂 町 ７ − ２
Target : 7 5 4 0 0 1 4
Predict : 7 5 4 0 0 1 4


epoch : 2 , batch : 104 , loss : 0.02
epoch : 2 , batch : 104 , accuracy : 98.00%
Source : 兵 庫 県 神 戸 市 長 田 区 大 日 丘 町 ３ 丁 目 ２ − １
Target : 6 5 3 0 8 7 2
Predict : 6 5 3 0 8 7 2


epoch : 2 , batch : 105 , loss : 0.01
epoch : 2 , batch : 105 , accuracy : 99.00%
Source : 埼 玉 県 草 加 市 清 門 １ 丁 目 ２ ７ ０ − １
Target : 3 4 0 0 0 5 5
Predict : 3 4 0 0 0 5 5


epoch : 2 , batch : 106 , loss : 0.02
epoch : 2 , batch : 106 , accuracy : 98.33%
Source : 徳 島 県 板 野 郡 藍 住 町 奥 野 字 原 ２ ２ ３
Target : 7 7 1 1


epoch : 2 , batch : 147 , loss : 0.02
epoch : 2 , batch : 147 , accuracy : 97.67%
Source : 沖 縄 県 名 護 市 大 北 ５ 丁 目 ２ − ７
Target : 9 0 5 0 0 1 9
Predict : 9 0 5 0 0 1 9


epoch : 2 , batch : 148 , loss : 0.01
epoch : 2 , batch : 148 , accuracy : 98.33%
Source : 福 島 県 郡 山 市 名 郷 田 １ 丁 目 １ ０
Target : 9 6 3 8 0 4 3
Predict : 9 6 3 8 0 4 3


epoch : 2 , batch : 149 , loss : 0.01
epoch : 2 , batch : 149 , accuracy : 98.67%
Source : 兵 庫 県 神 戸 市 長 田 区 大 塚 町 １ 丁 目 １ − １ ７
Target : 6 5 3 0 8 1 1
Predict : 6 5 3 0 8 1 1


epoch : 2 , batch : 150 , loss : 0.01
epoch : 2 , batch : 150 , accuracy : 97.67%
Source : 大 阪 府 大 阪 市 住 吉 区 南 住 吉 １ 丁 目 ２ ３ − １ ７
Target : 5 5 8 0 0 4 1
Predict : 5 5 8 0 0 4 1


epoch : 2 , batch : 151 , loss : 0.01
epoch : 2 , batch : 151 , accuracy : 98.67%
Source : 愛 知 県 名 古 屋 市 昭 和 区 緑 町 １ 丁 目 ２ ５ − １
Target : 4 6 6 0 0 1 3
Predict : 4 6 6 0 0 1 3


epoch : 2 , batch : 152 , loss : 0.01
epoch : 2 , batch : 152 , accuracy : 98.67%
Source : 兵 庫 県 神 戸 市 長 田 区 鹿 松 町 ２ 丁 目 ６ − ３



epoch : 2 , batch : 194 , loss : 0.01
epoch : 2 , batch : 194 , accuracy : 97.33%
Source : 群 馬 県 太 田 市 高 瀬 町 １ ０ ７ − ２
Target : 3 7 3 0 0 7 2
Predict : 3 7 3 0 0 7 2


epoch : 2 , batch : 195 , loss : 0.02
epoch : 2 , batch : 195 , accuracy : 97.67%
Source : 富 山 県 射 水 市 今 井 ２ ０ ８ − ６
Target : 9 3 4 0 0 4 6
Predict : 9 3 4 0 0 4 6


epoch : 2 , batch : 196 , loss : 0.01
epoch : 2 , batch : 196 , accuracy : 97.67%
Source : 愛 知 県 名 古 屋 市 昭 和 区 阿 由 知 通 ４ 丁 目 １ ８ − １ − ３ ０ １
Target : 4 6 6 0 0 2 7
Predict : 4 6 6 0 0 2 7


epoch : 2 , batch : 197 , loss : 0.01
epoch : 2 , batch : 197 , accuracy : 98.67%
Source : 石 川 県 金 沢 市 も り の 里 ２ 丁 目 １ ８ ８
Target : 9 2 0 1 1 6 7
Predict : 9 2 0 1 1 6 7


epoch : 2 , batch : 198 , loss : 0.01
epoch : 2 , batch : 198 , accuracy : 99.67%
Source : 東 京 都 足 立 区 竹 の 塚 ６ 丁 目 ７ − １ − １ ０ １
Target : 1 2 1 0 8 1 3
Predict : 1 2 1 0 8 1 3


epoch : 2 , batch : 199 , loss : 0.02
epoch : 2 , batch : 199 , accuracy : 98.00%
Source : 兵 庫 県 神 戸 市 長 田 区 長 田 町 ２ 丁 目 ４ − 


epoch : 2 , batch : 240 , loss : 0.01
epoch : 2 , batch : 240 , accuracy : 98.33%
Source : 栃 木 県 宇 都 宮 市 陽 東 ６ 丁 目 ４ − １ ７
Target : 3 2 1 0 9 0 4
Predict : 3 2 1 0 9 0 4


epoch : 2 , val loss : 0.02
epoch : 2 , val accuracy : 97.85%


epoch : 2 , batch : 241 , loss : 0.02
epoch : 2 , batch : 241 , accuracy : 97.33%
Source : 鹿 児 島 県 鹿 児 島 市 荒 田 ２ 丁 目 ２ ８ − １ ９
Target : 8 9 0 0 0 5 4
Predict : 8 9 0 0 0 5 4


epoch : 2 , batch : 242 , loss : 0.01
epoch : 2 , batch : 242 , accuracy : 98.67%
Source : 長 野 県 上 田 市 神 畑 ９ ４ ８ − ５
Target : 3 8 6 1 1 0 3
Predict : 3 8 6 1 1 0 3


epoch : 2 , batch : 243 , loss : 0.01
epoch : 2 , batch : 243 , accuracy : 97.67%
Source : 静 岡 県 藤 枝 市 大 手 １ 丁 目 ２ ７ − ２ １
Target : 4 2 6 0 0 1 7
Predict : 4 2 6 0 0 1 7


epoch : 2 , batch : 244 , loss : 0.02
epoch : 2 , batch : 244 , accuracy : 97.00%
Source : 茨 城 県 神 栖 市 土 合 中 央 ３ 丁 目 ７ − ４
Target : 3 1 4 0 3 4 7
Predict : 3 1 4 0 3 4 7


epoch : 2 , batch : 245 , loss : 0.01
epoch : 2 , batch : 245 , accuracy : 98


epoch : 2 , batch : 286 , loss : 0.01
epoch : 2 , batch : 286 , accuracy : 98.67%
Source : 北 海 道 釧 路 市 南 浜 町 ５ − ９
Target : 0 8 5 0 0 2 2
Predict : 0 8 5 0 0 2 2


epoch : 2 , batch : 287 , loss : 0.01
epoch : 2 , batch : 287 , accuracy : 99.00%
Source : 愛 知 県 名 古 屋 市 昭 和 区 鶴 舞 ２ 丁 目 １ ６ − １ ０
Target : 4 6 6 0 0 6 4
Predict : 4 6 6 0 0 6 4


epoch : 2 , batch : 288 , loss : 0.01
epoch : 2 , batch : 288 , accuracy : 97.67%
Source : 兵 庫 県 神 戸 市 長 田 区 若 松 町 ５ 丁 目 ５ − １
Target : 6 5 3 0 0 3 8
Predict : 6 5 3 0 0 3 8


epoch : 2 , batch : 289 , loss : 0.02
epoch : 2 , batch : 289 , accuracy : 97.00%
Source : 香 川 県 東 か が わ 市 湊 ６ ３ ４ − １ ０
Target : 7 6 9 2 7 0 1
Predict : 7 6 9 2 7 0 1


epoch : 2 , batch : 290 , loss : 0.01
epoch : 2 , batch : 290 , accuracy : 98.67%
Source : 群 馬 県 高 崎 市 足 門 町 ３ ９ − １ １
Target : 3 7 0 3 5 3 1
Predict : 3 7 0 3 5 3 1


epoch : 2 , batch : 291 , loss : 0.01
epoch : 2 , batch : 291 , accuracy : 98.67%
Source : 東 京 都 千 代 田 区 九 段 北 １ 丁 目 １ ２ − ４
Target : 1 0 2 0